In [ ]:
# BLOQUE 1: Importaciones y configuración inicial

import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd
import time
import re
from urllib.parse import urljoin, quote
import json

print("✅ Librerías importadas correctamente")
print("🕷️ Comenzando la infiltración en Books To Scrape...")


# BLOQUE 2: Funciones de Web Scraping 

def get_all_categories ():
    ''' OBTIENE TODAS LAS CATEGORIAS DEL SITIO'''
    url = "https://books.toscrape.com"
    respuesta = requests.get(url) # se descarga la web 
    soup = BeautifulSoup(respuesta.content, 'html.parser')  
    # convierte texto HTML ilegible, en un objeto inteligente que se pueda navegar 
    
    categories = []
    nav_list = soup.find('ul', class_='nav nav-list')
    if nav_list:
        category_links = nav_list.find_all ('a'[1:]) # Saltar "Books"
        for link in category_links:
            category_name = link.text.strip()
            category_url = urljoin(url, link['href'])
            categories.append({
                'name' : category_name,
                'url' : category_url
            })
            
    print (f"🎯 Encontradas {len(categories)} categorias")
    return categories 


def book_quantity (book_url): 
    ''' OBTIENE LA CANTIDAD EN STOCK DE UN LIBRO DESDE SU PAGINA DE DETALLES  '''
    try:
        soup_quantity = BeautifulSoup(requests.get(book_url).content,'html.parser') 
        quantity_text = soup_quantity.select_one('p.instock.availability').get_text(strip=True)
        match = re.search(r'\((\d+))', quantity_text)
        if match:
            return int (match.group(1)) # devuelve la cantidad encontrada 
        else:
            return 0 # si no se encuentra la cantidad, devuelve 0 
    except Exception as e :
        print (f"❌ Erorr obteniendo cantidad para {book_url}:{e}")
        return 0 # en caso de error, devuelve 0 
    


def scrape_books_from_page(page_url): 
    ''' SCRAPE LIBROS DE UNA PAGINA ESPECIFICA  '''
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content,'html.parser' )
    
    books = []
    book_containers = soup.find_all ('article', class_ ='product_pod')
    
    for book in book_containers: 
        try:
            # TITULO
            title_element = book.find('h3').find('a')
            title = title_element['title']

            # URL DEL LIBRO PARA MAS DETALLES 
            book_url = urljoin(page_url, title_element['href'])
            
            price_element = book.find('p', class_= 'price_color')
            price_text = price_element.text.strip() if price_element else "£0.00"
            price = float (price_text.lstrip('Â£'))
            
            # RATING 
            rating_element = book.find ('p', class_= 'star-rating')
            rating_class = rating_element['class'][1] if rating_element else 'Zero'
            rating_map = {'One': 1, 'Two' : 2 , 'Three': 3, 'Four': 4, 'Five': 5, "Zero" : 0 } 
            rating = rating_map.get(rating_class, 0)
            
            
            # STOCK 
            stock_element = book.find ('p', class_= 'instock availability')
            in_stock = 'In stock' in stock_element.text if stock_element else False 
            quantity = book_quantity(book_url)
            
            
            books.append({
                
                'title': title,
                'price': price,
                'rating' : rating,
                'in_stock' : in_stock,
                'quantity': quantity,
                'url': book_url
            })
            
        except Exception as e:
            print(f"❌ Error procesando libro: {e}")
            continue

In [ ]:

# BLOQUE 6: Diseño UML y Estructura de Base de Datos


print("""
🏗️ DIAGRAMA UML - Estructura de la Base de Datos

┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│     AUTORES     │    │  LIBROS_AUTORES  │    │     LIBROS      │
├─────────────────┤    │   (Tabla Pivot)  │    ├─────────────────┤
│ id (PK)         │◄───┤ autor_id (FK)    │───►│ id (PK)         │
│ nombre          │    │ libro_id (FK)    │    │ titulo          │
│                 │    └──────────────────┘    │ precio          │
└─────────────────┘                            │ rating           │
                                            │ categoria          │
┌─────────────────┐                           │ en_stock         │
│   CATEGORIAS    │                           │ fecha_publicacion│
├─────────────────┤                           │ descripcion      │
│ id (PK)         │◄──────────────────────────│ categoria_id (FK)│
│ nombre          │                           │ url              │
│                 │                           └─────────────────┘
└─────────────────┘

Relaciones:
- AUTORES ↔ LIBROS: Muchos a Muchos (un autor puede escribir varios libros, 
  un libro puede tener varios autores)
- CATEGORIAS ↔ LIBROS: Uno a Muchos (una categoría tiene muchos libros)
""")

In [ ]:
# Creacion de base de datos ( DDL) 